Import Libraries

In [ ]:
from pyspark.sql.functions import col, current_timestamp, expr

In [ ]:
Capture Synapse Variables

In [ ]:
vDay                    = dbutils.widgets.get("pDay")
vMonth                  = dbutils.widgets.get("pMonth")
vYear                   = dbutils.widgets.get("pYear")
vContainer              = dbutils.widgets.get("pContainer")
vStorageAccountName     = dbutils.widgets.get("pStorageAccountName")
vDatabase               = dbutils.widgets.get("pDatabase")
vDatabaseName           = dbutils.widgets.get("pDatabaseName")
vSchemaName             = dbutils.widgets.get("pSchemaName")
vTableName              = dbutils.widgets.get("pTableName")
vLayer                  = dbutils.widgets.get("pLayer")

In [ ]:
# vDay                    = "30"
# vMonth                  = "10"
# vYear                   = "2024"
# vContainer              = "raw"
# vStorageAccountName     = "stgeusproddata01"
# vDatabase               = "sqlserver"
# vDatabaseName           = "FCSCDC"
# vSchemaName             = "dbo"
# vTableName              = "EPEND"
# vLayer                  = "bronze"

Mount ADLS

In [ ]:
spark.conf.set(f"fs.azure.account.key.{vStorageAccountName}.dfs.core.windows.net",
                dbutils.secrets.get(scope = "key-vault-secrets", key = "token-storage-datalake"))

display(dbutils.fs.ls(f"abfss://{vContainer}@{vStorageAccountName}.dfs.core.windows.net/{vDatabase}/{vDatabaseName}/{vSchemaName}/{vTableName}/{vYear}/{vMonth}/{vDay}"))

Read Parquet File

In [ ]:
local_path = f"abfss://{vContainer}@{vStorageAccountName}.dfs.core.windows.net/{vDatabase}/{vDatabaseName}/{vSchemaName}/{vTableName}/{vYear}/{vMonth}/{vDay}"

df = spark.read.parquet(f"{local_path}/{vTableName}.parquet")

df_count = df.count()

Exit command if parquet if empty

In [ ]:
if df_count == 0:
    dbutils.notebook.exit("Succeeded")

Change SYS_CHANGE_VERSION and SYS_CHANGE_CREATION_VERSION to INT datatype

In [ ]:
df = df.withColumn("SYS_CHANGE_VERSION", col("SYS_CHANGE_VERSION").cast("int")).withColumn("SYS_CHANGE_CREATION_VERSION", col("SYS_CHANGE_CREATION_VERSION").cast("int"))

Add PARTITION_TIME column inside dataframe

In [ ]:
df = df.withColumn("PARTITION_TIME", expr("current_timestamp() - INTERVAL 3 HOURS"))

Save dataframe to delta table inside bronze layer

In [ ]:
df.write.format("delta").partitionBy("PARTITION_TIME").mode("append").option("mergeSchema", "true").saveAsTable(f"{vLayer}_{vDatabaseName}.{vTableName}")

Display lines delta table of bronze layer

In [ ]:
query = f"SELECT * FROM {vLayer}_{vDatabaseName}.{vTableName} LIMIT 10"
display_query = spark.sql(query)
display(display_query)